In [11]:
import os
import socket
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, length, when, col, size, array_contains, flatten, explode
from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from jinja2 import Environment, FileSystemLoader

In [12]:
# setting constants
SPARK_ADDRESS = "local[4]"
APP_NAME = "Lab1"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1"\
    .format(LOG4J_PROP_FILE)

In [13]:
# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

In [14]:
# run spark
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(SPARK_ADDRESS)\
    .config("spark.ui.port", "4040")\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.6")\
    .config("spark.driver.memory", "4g")\
    .config("spark.executor.memory", "6g")\
    .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
    .getOrCreate()

In [15]:
print("Web UI: {}".format(spark.sparkContext.uiWebUrl))
print("\nlog4j file: {}".format(LOG4J_PROP_FILE))
print("\ndriver log file: {}".format(LOG_FILE))

Web UI: http://jupyter-spark-7545d5766-kd8j5:4040

log4j file: /home/jovyan/nfs-home/conf/pyspark-log4j-Lab1.properties

driver log file: /home/jovyan/nfs-home/logs/pyspark-Lab1.log


In [16]:
df_posts = spark.read.json("file:///home/jovyan/shared-data/bigdata20/posts_api.json")

In [17]:
df_posts.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- created: long (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |-- size: long (nullable = true)
 |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- sizes: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |    |-- type: string (nullable = true)
 |   

## Find the top 20 posts in the group: (a) by likes; (b) by comments; (c) by reposts. 

In [18]:
df_posts\
    .orderBy(col("likes.count").desc())\
    .select(col("id").alias("post_id"), col("likes.count").alias("likes_count"))\
    .limit(20) \
    .toPandas()

,post_id,likes_count
0,32022,1637
1,35068,1629
2,17492,1516
3,18526,1026
4,19552,955
5,41468,952
6,19419,868
7,29046,824
8,32546,786
9,24085,765


In [19]:
df_posts\
    .orderBy(col("comments.count").desc())\
    .select(col("id").alias("post_id"), col("comments.count").alias("comments_count"))\
    .limit(20) \
    .toPandas()

,post_id,comments_count
0,24085,850
1,22540,250
2,27722,192
3,8285,148
4,26860,113
5,13571,107
6,39294,104
7,36680,96
8,26006,92
9,41739,92


In [20]:
df_posts\
    .orderBy(col("reposts.count").desc())\
    .select(col("id").alias("post_id"), col("reposts.count").alias("reposts_count"))\
    .limit(20) \
    .toPandas()

,post_id,reposts_count
0,17492,334
1,19552,246
2,32022,210
3,11842,129
4,19419,126
5,13532,110
6,17014,105
7,35068,101
8,41266,92
9,12593,90


## Find the top 20 users by (a) likes and (b) reposts they have made (to trace reposts use "copy_history" field) 

In [21]:
df_posts_likes = spark.read.parquet("file:///home/jovyan/shared-data/bigdata20/posts_likes.parquet")

In [22]:
df_posts_likes.head()

Row(itemType='post', ownerId=-94, itemId=13499, likerId=2070090)

In [73]:
df_posts_likes \
    .groupBy("likerId") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20) \
    .toPandas()

,likerId,count
0,2070090,4801
1,2397858,2055
2,1475301,1829
3,18239,1569
4,546612,1245
5,6371,907
6,1841959,746
7,78440957,709
8,120248,699
9,40981497,611


In [23]:
df_followers_posts = spark.read.json("file:///home/jovyan/shared-data/bigdata20/followers_posts_api_final.json")
df_followers_posts.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- created: long (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |-- size: long (nullable = true)
 |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- lat: double (nullable = true)
 |    |    |    |    |-- long: double (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- sizes: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    

In [9]:
df_followers_posts.schema.json()

'{"fields":[{"metadata":{},"name":"attachments","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"album","nullable":true,"type":{"fields":[{"metadata":{},"name":"created","nullable":true,"type":"long"},{"metadata":{},"name":"description","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"owner_id","nullable":true,"type":"long"},{"metadata":{},"name":"size","nullable":true,"type":"long"},{"metadata":{},"name":"thumb","nullable":true,"type":{"fields":[{"metadata":{},"name":"access_key","nullable":true,"type":"string"},{"metadata":{},"name":"album_id","nullable":true,"type":"long"},{"metadata":{},"name":"date","nullable":true,"type":"long"},{"metadata":{},"name":"id","nullable":true,"type":"long"},{"metadata":{},"name":"lat","nullable":true,"type":"double"},{"metadata":{},"name":"long","nullable":true,"type":"double"},{"metadata":{},"name":"owner_id","nullable":true,"type":"long"},{"

In [193]:
ITMO_GROUP_ID = -94

In [201]:
df_followers_posts \
    .withColumn("exploded_history", explode(col("copy_history"))) \
    .withColumn('group_id', col('exploded_history.from_id')) \
    .withColumn('post_id', col('exploded_history.id')) \
    .where(col("group_id") == ITMO_GROUP_ID) \
    .select(col("id").alias("repost_id"), col("from_id").alias("user_id")) \
    .groupBy("user_id") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20) \
    .toPandas()

,user_id,count
0,180907432,48
1,317799,16
2,4068532,13
3,2547211,9
4,484122052,8
5,1077823,5
6,281951154,5
7,268247082,5
8,2070090,5
9,527580876,4


## Get reposts of the original posts of the itmo group (posts.json) from user posts (the result should be similar to (group_post_id, Array (user_post_ids)))

In [202]:
df_followers_posts \
    .withColumn("exploded_history", explode(col("copy_history"))) \
    .withColumn('group_id', col('exploded_history.from_id')) \
    .withColumn('post_id', col('exploded_history.id')) \
    .where(col("group_id") == ITMO_GROUP_ID) \
    .select(col("post_id"), col("id").alias("repost_id")) \
    .groupBy(col("post_id")) \
    .agg(F.collect_list("repost_id").alias("repost_ids")) \
    .limit(20) \
    .toPandas()

,post_id,repost_ids
0,41424,"[4125, 529, 6941]"
1,42388,[11053]
2,39407,"[3536, 9214]"
3,39719,[1160]
4,38963,"[3720, 393, 6403, 814, 4868]"
5,40011,"[2559, 19267, 4770]"
6,39259,"[1751, 1205, 822, 10810, 1492, 14840, 10683, 3..."
7,40084,[536]
8,42009,"[120, 1509]"
9,41506,"[12823, 135, 670, 397, 398]"


## Find emoticons in posts (negative, positive, neutral), calculate their overall count,  frequency (number of posts they can be found in) and average count per post. Print top 10 most popular emoticons, print top 5 emoticons which have the greatest difference between their overall count and frequency, print top 5 emoticons with average count per post.

In [206]:
! pip install --user --trusted-host pypi-registry.supplementary-services.svc.cluster.local --index http://pypi-registry.supplementary-services.svc.cluster.local:8080/ emoji

Looking in indexes: http://pypi-registry.supplementary-services.svc.cluster.local:8080/
Note: you may need to restart the kernel to use updated packages.


In [6]:
import emoji
import re

emoji.emojize('Python is :thumbs_up:')

'Python is 👍'

In [41]:
from emoji import UNICODE_EMOJI

def find_all_emoji(text):
    text = emoji.demojize(text)
    text = re.findall(r':\w*:', text)
    list_emoji = [emoji.emojize(x) for x in text]
    
    if not list_emoji:
        return None
    else:
        return list_emoji
        

search_all_emoji = udf(lambda y: find_all_emoji(y), ArrayType(StringType()))

In [69]:
total_posts = df_posts.count()

result = df_posts \
    .where(col("text").isNotNull()) \
    .withColumn("emoji_in_post", search_all_emoji(col("text"))) \
    .select(col("emoji_in_post")) \
    .where(col("emoji_in_post").isNotNull()) \
    .withColumn("count", size(col("emoji_in_post"))) \
    .agg(F.count("emoji_in_post").alias("total_posts_with_emoji"), F.sum("count").alias("total_emoji")) \
    .toPandas()

total_posts_with_emoji, total_emoji = result["total_posts_with_emoji"][0], result["total_emoji"][0]

print('Total posts:', total_posts)
print('Total posts with emoji:', total_posts_with_emoji)
print('Total emoji:', total_emoji)
print('Average per posts:', total_emoji / total_posts)

Total posts: 18430
Total posts with emoji: 744
Total emoji: 1443
Average per posts: 0.07829625610417797


In [94]:
emoji_stat_df = df_posts \
    .where(col("text").isNotNull()) \
    .withColumn("emoji_in_post", search_all_emoji(col("text"))) \
    .select(col("emoji_in_post"), col("id")) \
    .where(col("emoji_in_post").isNotNull()) \
    .withColumn("emoji", explode(col("emoji_in_post"))) \
    .groupBy("emoji") \
    .agg(F.countDistinct("id").alias("post_count"), F.count("id").alias("count")) \
    .withColumn("average_per_post", col("count") / col("post_count"))

In [96]:
emoji_stat_df.orderBy(col("count").desc()).limit(10).toPandas()

,emoji,post_count,count,average_per_post
0,🔥,61,76,1.245902
1,⚡,45,68,1.511111
2,📍,39,63,1.615385
3,🚀,47,50,1.063830
4,❤,46,46,1.000000
5,❗,20,45,2.250000
6,🔵,17,40,2.352941
7,⬇,16,40,2.500000
8,✔,9,40,4.444444
9,💙,38,38,1.000000


In [98]:
emoji_stat_df \
        .withColumn("diff", col("count") - col("post_count")) \
        .orderBy(col("diff").desc()) \
        .limit(5).toPandas()

,emoji,post_count,count,average_per_post,diff
0,✔,9,40,4.444444,31
1,❗,20,45,2.250000,25
2,📍,39,63,1.615385,24
3,⬇,16,40,2.500000,24
4,🔵,17,40,2.352941,23


In [97]:
emoji_stat_df.orderBy(col("average_per_post").desc()).limit(5).toPandas()

,emoji,post_count,count,average_per_post
0,✔,9,40,4.444444
1,🔹,7,30,4.285714
2,▪,5,21,4.200000
3,✒,2,7,3.500000
4,🔘,1,3,3.000000


## Probable “fans”. For each user find the top 10 other users whose posts this user likes.

In [6]:
df_followers_posts_likes = spark.read.parquet("file:///home/jovyan/shared-data/bigdata20/followers_posts_likes.parquet")
df_followers_posts_likes.printSchema()

root
 |-- itemType: string (nullable = true)
 |-- ownerId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- likerId: integer (nullable = true)



In [8]:
df_followers_posts_likes.show()

+--------+-------+------+---------+
|itemType|ownerId|itemId|  likerId|
+--------+-------+------+---------+
|    post|   1567|  3324|     2767|
|    post|   1567|  3324|   254280|
|    post|   1567|  3324|   553430|
|    post|   1567|  3324|   905379|
|    post|   1567|  3324|  2411502|
|    post|   1567|  3324|  5274303|
|    post|   1567|  3324| 50038414|
|    post|   1567|  3324| 84207602|
|    post|   1567|  3324|145422426|
|    post|   1567|  3331|  4601081|
|    post|   1567|  3331|     1113|
|    post|   1567|  3331|  1324389|
|    post|   1567|  3331|  5274303|
|    post|   1567|  3332|   107645|
|    post|   1567|  3332|   899669|
|    post|   1567|  3332|  5274303|
|    post|   1567|  3332|  5516136|
|    post|   1567|  3332| 21842499|
|    post|   1567|  3332| 45086276|
|    post|   1567|  3332| 84207602|
+--------+-------+------+---------+
only showing top 20 rows



In [7]:
import operator

#https://stackoverflow.com/a/53346512
#https://stackoverflow.com/a/46584296

def sorter(l):
  res = sorted(l, key=operator.itemgetter(1), reverse=True)
  return res


sort_udf = F.udf(sorter, ArrayType(StructType([ \
    StructField("ownerId", IntegerType(), False), \
    StructField("count(itemId)", IntegerType(), False) \
])))

df_top_likers = df_followers_posts_likes \
    .groupBy(col("likerId"), col("ownerId")) \
    .agg(F.count("itemId")) \
    .groupBy(col("likerId")) \
    .agg(F.collect_list(F.struct("ownerId", "count(itemId)")).alias("unordered_liked")) \
    .select("likerId", sort_udf("unordered_liked").alias("most_liked")) \
    .select("likerId", F.expr("slice(most_liked, 1, 10)").alias("top_10_most_liked")) 

In [10]:
df_top_likers.show(truncate = False)

+-------+----------------------------------------------------+
|likerId|top_10_most_liked                                   |
+-------+----------------------------------------------------+
|496    |[{59139083, 1}]                                     |
|2142   |[{5411213, 5}, {152962, 1}]                         |
|3918   |[{145254284, 1}]                                    |
|7880   |[{2812004, 2}]                                      |
|9376   |[{111195, 1}]                                       |
|12046  |[{3824163, 2}]                                      |
|13832  |[{15221, 1}]                                        |
|18944  |[{15221, 1}]                                        |
|20135  |[{75791, 3}, {174291546, 1}]                        |
|20683  |[{591512, 1}]                                       |
|20924  |[{327458, 1}]                                       |
|23364  |[{131719318, 1}]                                    |
|26087  |[{482756, 1}]                                 

## Probable friends. If two users like each other posts they may be friends. Find pairs of users where both users are top likers of each other.

In [16]:
df_top_liked = df_top_likers.select(col("likerId"), col("top_10_most_liked.ownerId").getItem(0).alias("top_liked"))
df_top_liked.show()

+-------+---------+
|likerId|top_liked|
+-------+---------+
|    496| 59139083|
|   2142|  5411213|
|   3918|145254284|
|   7880|  2812004|
|   9376|   111195|
|  12046|  3824163|
|  13832|    15221|
|  18944|    15221|
|  20135|    75791|
|  20683|   591512|
|  20924|   327458|
|  23364|131719318|
|  26087|   482756|
|  26623|   216575|
|  28577|   471165|
|  30654|    55586|
|  32445|   715211|
|  40515|  2070090|
|  41751|   770885|
|  42468|     1087|
+-------+---------+
only showing top 20 rows



In [18]:
cond = col("l.likerId") == col("r.top_liked")

df_top_liked.alias("l").join(df_top_liked.alias("r"), cond, "inner") \
    .where(col("r.likerId") == col("l.top_liked")) \
    .where(col("l.likerId") != col("r.likerId")) \
    .select(col("l.likerId").alias("potential_friend_1"), col("r.likerId").alias("potential_friend_2")) \
    .show()

+------------------+------------------+
|potential_friend_1|potential_friend_2|
+------------------+------------------+
|         272076217|         209077977|
|           4428906|         126259762|
|          50236099|           8496525|
|           2964588|           5613498|
|          18737802|            135451|
|         135320883|          19948946|
|         173546700|          40147706|
|            638492|            362041|
|           9836958|         222900543|
|         134969149|          68495921|
|         234067084|         122236381|
|         162535930|          94967714|
|          25302595|         136217422|
|          31120049|          91100851|
|          68100112|          54944054|
|          78764885|           3583416|
|            476992|          24603721|
|          82028992|          52033806|
|         120462155|         165227372|
|         147035701|           1181958|
+------------------+------------------+
only showing top 20 rows



In [8]:
df_followers = spark.read.parquet("file:///home/jovyan/shared-data/bigdata20/followers.parquet")
df_followers.printSchema()

root
 |-- profile: integer (nullable = true)
 |-- follower: integer (nullable = true)



## Shutdown

In [24]:
spark.stop()